In [14]:
import wandb
import numpy as np
import torch
import torch.nn as nn

In [27]:
config = {
    'a': 10,
    'b': True,
    'c': 'hello'
}

wandb.init(name='hellowandb' ,config=config)
opt=wandb.config
wandb.finish

<function wandb.sdk.wandb_run.finish(exit_code: int = None, quiet: bool = None) -> None>

In [17]:
import yaml

with open('options.yml', encoding='UTF-8') as f:
    d = yaml.safe_load(f)

opt.update(d)

In [26]:
opt.as_dict()

{'epochs': 10,
 'learning_rate': 0.09820197101274794,
 'test_name': 'hello',
 'a': 10,
 'b': True,
 'c': 'hello',
 'project': 'GarbageClassification',
 'group': 'Resnet',
 'job_type': 'train',
 'config': {'n_epochs': 30,
  'batch_size': 16,
  'lr': 0.001,
  'weight_decay': 0.001},
 'dataroot': 'dataset/raw',
 'dataset_name': 'garbage-raw',
 'dataset_type': 'dataset',
 'dataset_desc': 'Raw garbage dataset',
 'dataset_metadata': {'num_classes': 6, 'train': 2024, 'test': 503},
 'model_name': 'resnet101',
 'model_desc': 'resnet101 + lr_cosine_annealing',
 'model_metadata': {'use_BN': True, 'activation': 'relu', 'dropout': 0.5}}

In [22]:
wandb.finish()

In [ ]:
sweep_config = {
    "name": "my-sweep",
    "method": "random",
    "metric": {
        "goal": 'minimize',
        "name": 'loss'
    },
    "parameters": {
        "epochs": {
            "values": [10, 20, 50]
        },
        "learning_rate": {
            "min": 0.0001,
            "max": 0.1
        },
        "test_name": {"value": 'hello'},
    }
}

sweep_id = wandb.sweep(sweep_config)

def train():
    with wandb.init() as run:
        config = wandb.config
        print(config)
        for epoch in range(config["epochs"]):
            loss = np.cos(epoch)
            wandb.log({"loss": loss, "epoch": epoch})

count = 5  # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

In [4]:
run = wandb.init(project="table-test")
my_table = wandb.Table(columns=["a", "b"], data=[["a1", "b1"], ["a2", "b2"]])
run.log({"Table Name": my_table})

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


In [12]:
# create a Table with the same columns as above,
# plus confidence scores for all labels
columns = ["id", "image", "guess", "truth"]
for digit in range(10):
    columns.append("score_" + str(digit))
test_table = wandb.Table(columns=columns)

# run inference on every image, assuming my_model returns the
# predicted label, and the ground truth labels are available
score = torch.linspace(0, 1, 10)
imgs = ['dataset/train/glass/glass' + str(i) + '.jpg' for i in range(3, 7)]
for img_id, img in enumerate(imgs):
    true_label = 1
    guess_label = 0
    test_table.add_data(img_id, wandb.Image(img), guess_label, true_label, *score.tolist())

wandb.log({"ttable": test_table})

In [13]:
wandb.finish()